In [ ]:
%matplotlib inline


# CRUD operations on Postgres tables using Neurobooth-terra

This example demonstrates how to perform `CRUD`_ operations
on postgres tables with neurobooth-terra.


In [ ]:
# Authors: Mainak Jas <mjas@harvard.mgh.edu>

Let us first import the necessary functions.



In [ ]:
from neurobooth_terra import list_tables, create_table, drop_table, Table

import psycopg2
import pandas as pd

Then, we will create a connection using ``psycopg2``.



In [ ]:
connect_str = ("dbname='neurobooth' user='neuroboother' host='localhost' "
               "password='neuroboothrocks'")

conn = psycopg2.connect(connect_str)

We will drop tables if they already exist
this is just for convenience so we can re-run this script
even when changing some columns



In [ ]:
drop_table('subject', conn)
drop_table('contact', conn)
drop_table('consent', conn)

## Create
Now we define the Table



In [ ]:
table_id = 'subject'
column_names = ['subject_id', 'first_name_birth', 'last_name_birth']
dtypes = ['VARCHAR (255)', 'VARCHAR (255)', 'VARCHAR (255)']
table_subject = create_table(table_id, conn,
                             column_names=column_names,
                             dtypes=dtypes)

and insert some data and retrieve the table as a dataframe



In [ ]:
table_subject.insert_rows([('x5dc', 'mainak', 'jas'),
                           ('y5d3', 'anoopum', 'gupta')],
                           cols=column_names)

We can create another table and relate it to the other table using
a foreign key



In [ ]:
table_id = 'contact'
column_names = ['subject_id', 'email']
dtypes = ['VARCHAR (255)', 'VARCHAR (255)']
table_contact = create_table(table_id, conn,
                             column_names=column_names,
                             dtypes=dtypes,
                             foreign_key=dict(subject_id='subject'))
table_contact.insert_rows([('x5dc',), ('y5d3',)], cols=['subject_id'])

## Read
We can get a list of tables in the database by doing



In [ ]:
list_tables(conn)

From the list of these table, we can select one to inspect by
creating a :class:`~neurobooth_terra.Table` object.



In [ ]:
table_id = 'subject'
table_subject = Table(table_id, conn)
print(table_subject)

With this Table object, we can create a query that
returns a dataframe



In [ ]:
df_subject = table_subject.query()
print(df_subject)

## Update
We can make changes such as adding a new column



In [ ]:
table_subject.add_column('dob', 'VARCHAR (255)')
print(table_subject.query())

To update a row in the table we can do



In [ ]:
table_subject.update_row('y5d3',
                         cols=['first_name_birth', 'last_name_birth'],
                         vals=('anupum', 'gupta'))
print(table_subject.query())

## Delete
We can also delete rows in our table



In [ ]:
table_subject.delete_row(condition="subject_id = 'yd53'")
print(table_subject.query())

Or drop columns



In [ ]:
table_contact.drop_column('subject_id')
print(table_subject.query())

To delete an entire table, we can do



In [ ]:
drop_table('subject', conn)
list_tables(conn)

Don't forget to close the connection once done!



In [ ]:
table_subject.close()
table_contact.close()
conn.close()